- For each candidate move a, calculate V_opt(a)
- Do a softmax to turn into probabilities, p(a) \propto exp(beta * V_opt(a)). For example with beta = 1.
- Calculate the entropy of that distribution. This is a measure of difficulty

- alternative metrics: difficulty = V_opt_max - V_opt(second best action)

- difficulty = average of (V_opt_max - V_opt(a)) for all candidate actions a

In [1]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import sem
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import ttest_rel,ttest_ind,wilcoxon
import json
import statsmodels.api as sm
import pylab as py

In [2]:
# home_dir = '/Users/dbao/google_drive_db'+'/road_construction/data/2022_online/'
# home_dir = '../../../2022_online/'
home_dir = '/Users/dbao/Library/CloudStorage/GoogleDrive-sherrydbao@gmail.com/.shortcut-targets-by-id/1sYZNVBbYCrHcHjo-9xdKcA4wLnob9ZEq/'+'/road_construction/data/2022_online/'
map_dir = 'active_map/'
data_dir  = 'data/preprocessed'
# out_dir = home_dir + 'figures/cogsci_2022/'
out_dir = './figures_dk/'
R_out_dir = home_dir + 'R_analysis_data/'

In [3]:
data_puzzle_level = pd.read_csv(R_out_dir +  'data.csv')
puzzleID_order_data = data_puzzle_level.sort_values(["subjects","puzzleID"])
data_choice_level = pd.read_csv(R_out_dir +  'choice_level/choicelevel_data.csv')

single_condition_data = puzzleID_order_data[puzzleID_order_data['condition']==1].copy()
single_condition_data = single_condition_data.reset_index()
sc_data_choice_level = data_choice_level[data_choice_level['condition']==1].reset_index()

- For each candidate move a, calculate V_opt(a)
- Do a softmax to turn into probabilities, p(a) \propto exp(beta * V_opt(a)). For example with beta = 1.
- Calculate the entropy of that distribution. This is a measure of difficulty


In [4]:
with open(home_dir + map_dir + 'tree/map_tree', 'r') as file:
    map_tree = json.load(file)

with open(home_dir + map_dir + '/basicMap.json', 'r') as file:
    map = json.load(file)
    
    

In [5]:
map

[{'N': 30,
  'total': 300,
  'R': 40000,
  'r': [18982.155991132306,
   26990.116554045977,
   29266.131786108195,
   6839.055070905809,
   2063.3558417720856,
   2557.945669618258,
   35762.392061821636,
   5878.185532928981,
   9291.704998676767,
   11979.02868213077,
   20861.428631246352,
   10104.594529867396,
   370.071962068943,
   6286.010530444677,
   4835.031884066341,
   5845.824428837712,
   30084.980971066165,
   2865.8847900062055,
   37889.392540550696,
   13143.263902125194,
   8318.457564931188,
   6783.337557139482,
   19121.375552396137,
   6876.605251334906,
   3082.6485790863912,
   26730.272653767715,
   1605.3327527372296,
   38371.19466097632,
   1195.7817679879668,
   26978.85264274518],
  'phi': [1.3906969142684467,
   4.519419210448505,
   3.3487273404459437,
   4.2374200916668014,
   3.2888507610062527,
   2.7350422614845797,
   1.1325905653207466,
   3.9486396921260694,
   3.9358423548806423,
   3.3483054484454255,
   4.681038599197451,
   2.955518922119199

In [8]:
difficulty_average_diff = []

from copy import deepcopy
map_action_gap = deepcopy(map)

for pzi in range(46):
    # pzi_data = sc_data_choice_level[sc_data_choice_level['puzzleID']==pzi] seems unnecessary
    
    TS = map[pzi]
    TS_opt = map_tree[pzi]
    # DQ: paths include all the possible states in the map
    TS_opt_mas = [TS_opt['mas'][i] for i in range(len(TS_opt['paths'])) if len(TS_opt['paths'][i]) == 2]
    
    # if pzi_data['currMas'].iloc[0] == np.max(TS_opt['mas']): seems unnecessary
    action_gap = np.mean(np.max(TS_opt_mas)-np.array(TS_opt_mas))
    # find the second best action among TS_opt_mas, and take the difference between the best and second best
    action_gap2 = np.max(TS_opt_mas)-np.sort(TS_opt_mas)[-2]
    print(action_gap2)

    map_action_gap[pzi]['action_gap'] = action_gap
    map_action_gap[pzi]['action_gap2'] = action_gap2
    difficulty_average_diff.append(action_gap)



0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
1
0
0
0
1
1
0
0
0
1
0
0
0
0
1
1
1
0
1
1
1
1
1
0
1
0
1
0
1
0
0


In [7]:
print(map_action_gap[0].keys())

with open(home_dir + map_dir +'map_action_gap.json', 'w') as file:
    json.dump(map_action_gap, file)

dict_keys(['N', 'total', 'R', 'r', 'phi', 'x', 'y', 'xy', 'city_start', 'distance', 'action_gap', 'action_gap2'])


TypeError: Object of type int64 is not JSON serializable